### Requirements

#### Windows
* Python 3 (includes pip) - run the installer from https://www.python.org/downloads/release/python-365/
* Add python to Path environment variable
* Pymarc 
    - from command prompt, ```pip install pymarc```

##### oddities
* the SavetoMRK function may encounter encoding errors even with to_unicode on a few characters. Even if this happens, the mrc file should be created with no problems. 

#### Mac/Linux
* Python 3 (includes pip)
* Pymarc 
    - from Terminal ```pip3 install pymarc```


In [1]:
import pymarc
from pymarc import MARCReader, Record, Field, marcxml, MARCWriter, TextWriter
import re
from io import BytesIO
from utilities import utilityFunctions 

utilities = utilityFunctions()

### The files

##### batch_cave_scratchpad.py

* Stripped-down version of command line batchcave application
* Contains a function template and one example
* Requires utilities.py to be in the same directory to run

##### utilities.py

* Slightly adapted for command line interface
* 2 major differences are 
    - SavetoMRK function: 
        - writes directly to text file
        - skips records that encounter encoding errors and prints the record number
    - MakeMARCFile function:
        - writes directly to binary mrc file
        - encodes as utf-8 if encoding errors are encountered
* The rest of the functions are the same for the web interface

##### batch_cave1.9_linux_py3.py
* Recreation in Python3 of https://github.com/crowesn/batch_cave/blob/master/1.9_RLWG_BatchEdit.py
* Operates on records, not strings. Requires PyMARC, not MARCEdit.
* Once a function works in the scratchpad, it can be integrated here.
* This provides a semi-complete command line application.
* To integrate in the web interface, a few minor edits are needed

In [2]:
# Start in batch_cave_scratchpad.py
# define a function inside class batchEdits
class batchEdits:
# pass self because it belongs to the batchEdits class
# x is a list of records
    def ER_MY_OPERATION(self, x, name='My Operation'):
        # give it an optional print statement
        print("Running change script " + name + "\n")
    

In [3]:
# the below is only for this demonstration
# not needed in batch_cave_scratchpad
with open('SAGE_Research_Methods_Books_Reference.mrc', 'rb') as fh:
    #create a MARCReader object with the file
    reader = MARCReader(fh)
    #create a blank list to hold the record objects
    records = []
    for rec in reader:
        #print(rec)
        records.append(rec)
  
BatchEdits = batchEdits()
BatchEdits.ER_MY_OPERATION(records)

Running change script My Operation



In [4]:
# new function to delete one field and create another

# define a function inside class batchEdits - this should not be changed in template
class batchEdits2:
# pass self because it belongs to the batchEdits class
# x is a file to read records from
    def ER_MY_SECOND_OPERATION(self, x, name='My 2nd Operation'):
        # give it an optional print statement
        print("Running change script " + name + "\n")
        # break the list of records in to individual records
        recs = utilities.BreakMARCFile(x)
        
        # inspect a record
        print(recs[0])
        
        # how many records?
        print(len(recs))
        
        # operate on each record individually
        for rec in recs:
            print(rec.title())
            #put value of 001 in 002
            field001 = rec['001']
            rec.add_ordered_field(Field(tag='002', data = field001.value()))
            #remove 001
            # since there could be multiple 001 fields (at least in PyMARC), we need to specify the 0th field should be removed
            rec.remove_field(rec.get_fields('001')[0])
      
            ## include stanard cleanup operations
            rec = utilities.DeleteLocGov(rec)
            rec = utilities.Standardize856_956(rec)
            
        # perform file-level operations after the loop has finished
        rec = utilities.SaveToMRK(recs, filename)
        rec = utilities.MakeMARCFile(recs, filename)
        
        # inspect results
        print(recs[0])
        
BatchEdits2 = batchEdits2()     
filename = 'SAGE_Research_Methods_Books_Reference.mrc'
BatchEdits2.ER_MY_SECOND_OPERATION('SAGE_Research_Methods_Books_Reference.mrc')

Running change script My 2nd Operation


<Breaking MARC file>

=LDR  01530nam a22003131i 4500
=001  SAGE000001379
=003  CaToSAGE
=005  20170126165630.4
=006  m|||||o||d||||||||
=007  cr\|||||||||||
=008  170126s2016\\\\caua\\\fo\\\\\000\0\eng\d
=020  \\$a9781526408495 (ebook) :$cNo price
=040  \\$aCaToSAGE$beng$cCaToSAGE$erda$epn$dUtOrBLW
=050  \4$aH62$b.B97 2016
=082  04$a300.72
=100  1\$aByrne, David,$eauthor.
=245  10$aPhilosophy of research /$cDavid Byrne.
=264  \1$aLos Angeles, CA :$bSAGE Publications, Inc.,$c2016.
=300  \\$a1 online resource :$billustrations.
=336  \\$atext$btxt$2rdacontent
=337  \\$acomputer$bc$2rdamedia
=338  \\$aonline resource$bcr$2rdacarrier
=490  1\$aSAGE Research Methods. Project Planner
=520  8\$aThis section deals with the philosophy of research and research methodology. Methodology underpins all the work you do. These are important concepts which need to be understood, but they're one of the areas people find most challenging, so don't worry if they don

Writing up qualitative research
GIS geographic information systems for the social sciences : investigating space and place /
Advances in social network analysis research in the social and behavioral sciences /
Focus group interviews in education and psychology
Pocket glossary for commonly used research terms
Multivariate general linear models
Using Stata for quantitative analysis
Best practices in data cleaning a complete guide to everything you need to do before and after collecting your data /
Feminist measures in survey research
Interactive evaluation practice mastering the interpersonal dynamics of program evaluation /
Methods of randomization in experimental design
Building SPSS graphs to understand data
Running behavioral experiments with human participants : a practical guide /
Social network analysis and education : theory, methods & applications /
Heteroskedasticity in regression : detection and correction /
An introduction to exponential random graph modeling /
Developing a l


<Compiling file to MARC>

=LDR  01530nam a22003131i 4500
=002  SAGE000001379
=003  CaToSAGE
=005  20170126165630.4
=006  m|||||o||d||||||||
=007  cr\|||||||||||
=008  170126s2016\\\\caua\\\fo\\\\\000\0\eng\d
=020  \\$a9781526408495 (ebook) :$cNo price
=040  \\$aCaToSAGE$beng$cCaToSAGE$erda$epn$dUtOrBLW
=050  \4$aH62$b.B97 2016
=082  04$a300.72
=100  1\$aByrne, David,$eauthor.
=245  10$aPhilosophy of research /$cDavid Byrne.
=264  \1$aLos Angeles, CA :$bSAGE Publications, Inc.,$c2016.
=300  \\$a1 online resource :$billustrations.
=336  \\$atext$btxt$2rdacontent
=337  \\$acomputer$bc$2rdamedia
=338  \\$aonline resource$bcr$2rdacarrier
=490  1\$aSAGE Research Methods. Project Planner
=520  8\$aThis section deals with the philosophy of research and research methodology. Methodology underpins all the work you do. These are important concepts which need to be understood, but they're one of the areas people find most challenging, so don't worry if they don't sink in right away. Read the sect

In [8]:
## Start from the template

#batchEdits
##########################################
class batchEdits3:
#keep everything above this
###########

    #### Keep everything from here to next comment ########
    def TEMPLATE_FUNCTION(self, x, name='Function Name'):
        print('\nRunning change script '+ name + '\n')
        recs = utilities.BreakMARCFile(x)
        for rec in recs:
        ##### Keep everything above this comment ########
        ##### Make changes to each record below ########
            # Change =001 field to =002, and add 003
            # rec.add_ordered_field(Field(tag = '002',data = rec['001'].value()))
            # remove 001
            # rec.remove_field(rec.get_fields('001')[0])
            ###### Keep everything below this ########
            rec = utilities.DeleteLocGov(rec)
            rec = utilities.Standardize856_956(rec)
        rec = utilities.SaveToMRK(recs, filename)
        x = utilities.MakeMARCFile(recs, filename)
        print(recs[0])
        #return x
        #### Keep everything above this ##########

BatchEdits3 = batchEdits3()     
filename = 'SAGE_Research_Methods_Books_Reference.mrc'
BatchEdits3.TEMPLATE_FUNCTION('SAGE_Research_Methods_Books_Reference.mrc')


Running change script Function Name


<Breaking MARC file>


<Writing file to MRK>


<Compiling file to MARC>

=LDR  01530nam a22003131i 4500
=001  SAGE000001379
=003  CaToSAGE
=005  20170126165630.4
=006  m|||||o||d||||||||
=007  cr\|||||||||||
=008  170126s2016\\\\caua\\\fo\\\\\000\0\eng\d
=020  \\$a9781526408495 (ebook) :$cNo price
=040  \\$aCaToSAGE$beng$cCaToSAGE$erda$epn$dUtOrBLW
=050  \4$aH62$b.B97 2016
=082  04$a300.72
=100  1\$aByrne, David,$eauthor.
=245  10$aPhilosophy of research /$cDavid Byrne.
=264  \1$aLos Angeles, CA :$bSAGE Publications, Inc.,$c2016.
=300  \\$a1 online resource :$billustrations.
=336  \\$atext$btxt$2rdacontent
=337  \\$acomputer$bc$2rdamedia
=338  \\$aonline resource$bcr$2rdacarrier
=490  1\$aSAGE Research Methods. Project Planner
=520  8\$aThis section deals with the philosophy of research and research methodology. Methodology underpins all the work you do. These are important concepts which need to be understood, but they're one of the areas people 

In [12]:
## Start from the template

#batchEdits
##########################################
class batchEdits4:
#keep everything above this
###########

    #### Keep everything from here to next comment ########
    ### change the name
    def SAGE_TEST(self, x, name='Sage Test'):
        print('\nRunning change script '+ name + '\n')
        recs = utilities.BreakMARCFile(x)
        for rec in recs:
        ##### Keep everything above this comment ########
        ##### Make changes to each record below ########
            # Change =001 field to =002, and add 003
            rec.add_ordered_field(Field(tag = '002',data = rec['001'].value()))
            # remove the 0th 001 field
            rec.remove_field(rec.get_fields('001')[0])
            
            ###### Keep everything below this ########
            rec = utilities.DeleteLocGov(rec)
            rec = utilities.Standardize856_956(rec)
        rec = utilities.SaveToMRK(recs, filename)
        x = utilities.MakeMARCFile(recs, filename)
        print(recs[0])
        #return x
        #### Keep everything above this ##########

BatchEdits4 = batchEdits4()     
filename = 'SAGE_Research_Methods_Books_Reference.mrc'
BatchEdits4.SAGE_TEST('SAGE_Research_Methods_Books_Reference.mrc')


Running change script Sage Test


<Breaking MARC file>


<Writing file to MRK>


<Compiling file to MARC>

=LDR  01530nam a22003131i 4500
=002  SAGE000001379
=003  CaToSAGE
=005  20170126165630.4
=006  m|||||o||d||||||||
=007  cr\|||||||||||
=008  170126s2016\\\\caua\\\fo\\\\\000\0\eng\d
=020  \\$a9781526408495 (ebook) :$cNo price
=040  \\$aCaToSAGE$beng$cCaToSAGE$erda$epn$dUtOrBLW
=050  \4$aH62$b.B97 2016
=082  04$a300.72
=100  1\$aByrne, David,$eauthor.
=245  10$aPhilosophy of research /$cDavid Byrne.
=264  \1$aLos Angeles, CA :$bSAGE Publications, Inc.,$c2016.
=300  \\$a1 online resource :$billustrations.
=336  \\$atext$btxt$2rdacontent
=337  \\$acomputer$bc$2rdamedia
=338  \\$aonline resource$bcr$2rdacarrier
=490  1\$aSAGE Research Methods. Project Planner
=520  8\$aThis section deals with the philosophy of research and research methodology. Methodology underpins all the work you do. These are important concepts which need to be understood, but they're one of the areas people find

In [15]:
## Complete SAGE example
## Start from the template

#batchEdits
##########################################
class batchEdits5:
#keep everything above this
###########

    #### Keep everything from here to next comment ########
    ### change the name
    def SAGE_COMPLETE(self, x, name='Sage Complete'):
        print('\nRunning change script '+ name + '\n')
        recs = utilities.BreakMARCFile(x)
        for rec in recs:
        ##### Keep everything above this comment ########
        ##### Make changes to each record below ########
            # Change =001 field to =002, and add 003
            rec.add_ordered_field(Field(tag = '002',data = rec['001'].value()))
            # do not remove the 0th 001 field
            # rec.remove_field(rec.get_fields('001')[0])
            rec.add_ordered_field(Field(tag = '949', indicators = ['\\', '1'],subfields = ['l','olink', 'r', 's', 't', '99']))
            rec.add_ordered_field(Field(tag = '949', indicators = ['\\', '\\'],subfields = ['a','*bn=bolin;']))
            rec.add_ordered_field(Field(tag = '730', indicators = ['0','\\'],subfields = ['a','Sage eReference..', '5', 'OCU']))
            rec.add_ordered_field(Field(tag = '003',data = 'ER-O/L-Sage-eRef'))
            rec.add_ordered_field(Field(tag = '002',data = 'O/L-Sage-eRef'))
            rec = utilities.AddEresourceGMD(rec)
            
            ###### Keep everything below this ########
            rec = utilities.DeleteLocGov(rec)
            rec = utilities.Standardize856_956(rec)
        rec = utilities.SaveToMRK(recs, filename)
        x = utilities.MakeMARCFile(recs, filename)
        print(recs[0])
        #return x
        #### Keep everything above this ##########

BatchEdits5 = batchEdits5()     
filename = 'SAGE_Research_Methods_Books_Reference.mrc'
BatchEdits5.SAGE_COMPLETE('SAGE_Research_Methods_Books_Reference.mrc')


Running change script Sage Complete


<Breaking MARC file>


<Writing file to MRK>


<Compiling file to MARC>

=LDR  01530nam a22003131i 4500
=001  SAGE000001379
=002  SAGE000001379
=002  O/L-Sage-eRef
=003  CaToSAGE
=003  ER-O/L-Sage-eRef
=005  20170126165630.4
=006  m|||||o||d||||||||
=007  cr\|||||||||||
=008  170126s2016\\\\caua\\\fo\\\\\000\0\eng\d
=020  \\$a9781526408495 (ebook) :$cNo price
=040  \\$aCaToSAGE$beng$cCaToSAGE$erda$epn$dUtOrBLW
=050  \4$aH62$b.B97 2016
=082  04$a300.72
=100  1\$aByrne, David,$eauthor.
=245  10$aPhilosophy of research /$h[electronic resource]
=264  \1$aLos Angeles, CA :$bSAGE Publications, Inc.,$c2016.
=300  \\$a1 online resource :$billustrations.
=336  \\$atext$btxt$2rdacontent
=337  \\$acomputer$bc$2rdamedia
=338  \\$aonline resource$bcr$2rdacarrier
=490  1\$aSAGE Research Methods. Project Planner
=520  8\$aThis section deals with the philosophy of research and research methodology. Methodology underpins all the work you do. These are important co